## Keyword Ordering
This code analyses text documenst and imports excel files. It then divides then creates frequency counts based on previously developed keywords.

In [ ]:
import pandas as pd
from PyPDF2 import PdfReader
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import os
import glob




def get_tokens(pdf_path):
    # Read PDF file and preprocess text
    # pdf_path = "/Users/jz/Desktop/CMU/lyx_pdfReader/Nexteer_report.pdf"
    pdf_file = open(pdf_path, 'rb')
    pdf_reader = PdfReader(pdf_file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    pdf_file.close()
    tokens = word_tokenize(text.lower())
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    return filtered_tokens

def collect(filepath):
    pdf_files = glob.glob(os.path.join(filepath, "*.pdf"))
    collections = {}
    for file in pdf_files:
        splits = file.split('/')
        filename = splits[len(splits)-1].split('_')[0]
        collections[filename] = get_tokens(file)
    return collections



# Read Excel file and create keyword sets based on categories
category_df = pd.read_excel("/Users/jenniferlaura/Documents/CMU - MISM-12/Spring 2023/Capstone Project/keywords.xlsx", sheet_name='Weights')    ## PATH TO THE WEIGHTS DOCUMENT
keywords_df = pd.read_excel("/Users/jenniferlaura/Documents/CMU - MISM-12/Spring 2023/Capstone Project/keywords.xlsx", sheet_name='Keywords')   ## PATH TO THE KEYWORDS DOCUMENT

# Path of the folder containing pdfs
filepath = "/Users/jenniferlaura/Documents/CMU - MISM-12/Spring 2023/Capstone Project/pdfs"


# get all tokens for each pdf file in the filepath
weight_dict = {}
subcategory_dict = {}
keywords_dict = {}

for i in range(len(category_df.columns) // 2):
    category_name = category_df.columns[i * 2]
    sub_category = category_df[category_df.columns[i * 2]].dropna().tolist()
    weight = category_df[category_df.columns[i * 2 + 1]].dropna().tolist()
    map = {k:v for k,v in zip(sub_category, weight)}
    weight_dict[category_name] = map
    for sub in sub_category:
        subcategory_dict[sub] = category_name

for i in range(len(keywords_df.columns)):
    category = keywords_df.columns[i]
    keywords = keywords_df[category].dropna().tolist()
    for keyword in keywords:
        keywords_dict[keyword] = category

collections = collect(filepath)
columnname = ["keyword", "subcategory", "category"]
for companyname in collections.keys():
    columnname.extend([companyname+"_freq", companyname + "_freq_weight"])
df1 = pd.DataFrame(columns = columnname)
lemmatizer = WordNetLemmatizer()
for keyword, subcategory in keywords_dict.items():
    category = subcategory_dict[subcategory]
    dict = {"keyword":keyword, "subcategory":subcategory, "category":category}
    for companyname, tokens in collections.items():
        frequency = 0
        words = word_tokenize(keyword.lower())
        for i in range(len(tokens) - len(words) + 1):
            if all(tokens[i + j] == lemmatizer.lemmatize(words[j]) for j in range(len(words))):
                frequency += 1
        freq_col = companyname + "_freq"
        freq_weight_col = companyname + "_freq_weight"
        dict[freq_col] = frequency
        dict[freq_weight_col] = frequency * weight_dict[category][subcategory]
    df1.loc[len(df1)] = dict

#df1 = df1.sort_values("category", ascending=True)
df1 = df1.sort_values(['category','subcategory'])

companys_freq_sum = {name+"_freq": "sum" for name in collections.keys()}
df2 = df1.groupby(["subcategory", "category"]).agg(companys_freq_sum).reset_index()
df2 = df2.sort_values("category", ascending=True)
df2_transpose = df2.transpose()
df2_transpose = df2_transpose.rename(columns=df2_transpose.iloc[0]).drop(df2_transpose.index[0])

company_freq_weight_sum = {name+"_freq_weight":"sum" for name in collections.keys()}
df3 = df1.groupby(["category"]).agg(company_freq_weight_sum).reset_index()
df3 = df3.sort_values("category", ascending=True)
df3_transpose = df3.transpose()
df3_transpose = df3_transpose.rename(columns=df3_transpose.iloc[0]).drop(df3_transpose.index[0])

df1.to_csv("StatisticChart.csv", index=False)
df2_transpose.to_csv("Freq.csv", index=True)
df3_transpose.to_csv("WeightedFreq.csv", index=True)